# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-1c86b16009-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Politics Workflow Series ("Politicians in E.U.") 

Consider the following exploratory information need:

> You investigating the careers of politicians in the E.U.

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wdt:P39`     | position held   | predicate |
| `wdt:P106`    | profession   | predicate |
| `wd:Q82955`   | politician | node      |
| `wd:Q46`      | Europe        | node |
| `wd:Q38`      | Italy          | node |



Also consider

```
?p wdt:P106/wdt:P279* wd:Q82955  . 
```

is the BGP to retrieve all **politicians**

## Workload Goals


1. Identify the BGP to retrieve E.U. countries and their politicians

2. Identify the BGP for obtaining other occupations and properties of politicians

3. How many politicians are recorder for each E.U. country?

4. Are there politicians with double citizenship?

5. Analyze the number of politicians in each country by occupation, for instance
 
   5.1 What are the top-3 occupations for a politician in Italy and France?
   
   5.2 What if you consider only politicians for which we don't have a date of death?
   
   5.3 Which politicians had a spouse that was also a politician? How many in each country?


In [2]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT(*)
WHERE { 

?p wdt:P106/wdt:P279 wd:Q82955  . 
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '15556')]


1

# Task 1

*Identify the BGP to retrieve E.U. countries and their politicians*

For completing the task, first we have to identify the node representing the E.U., so let's start by analyzing the properties that enter Europe as a continent:

In [ ]:
# Check connections to europe
queryString = """
SELECT (count(*)as ?p)
WHERE { 

?x ?p wd:Q46  . 
} 
"""

print("Results")
run_query(queryString)

Let's check in particular how is Italy connected to Europe

In [5]:
# check connections between Europe and Italy
queryString = """
SELECT (count(*)as ?p)
WHERE { 

wd:Q38 ?p wd:Q46  . 
}  
"""

print("Results")
run_query(queryString)

Results
[('p', '2')]


1

In [6]:
# list connections between europe and italy
queryString = """
SELECT ?p
WHERE { 

wd:Q38 ?p wd:Q46  . 
}  
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P30')]
[('p', 'http://www.wikidata.org/prop/direct/P361')]


2

In [7]:
# give a name to the connections between europe and italy
queryString = """
SELECT ?name
WHERE { 

wd:Q38 ?p wd:Q46  . 
?p <http://schema.org/name> ?name
}  
"""

print("Results")
run_query(queryString)

Results
[('name', 'continent')]
[('name', 'part of')]


2

Italy is "part of" Europe, so maybe it will be part of the E.U.

In [8]:
# "part of" seems promising, try to see if maybe italy is "part of" the EU other than the continent Europe
queryString = """
SELECT (count(?name) as ?x)
WHERE { 

wd:Q38 wdt:P361 ?name  . 

}  
"""

print("Results")
run_query(queryString)

Results
[('x', '1')]


1

Italy is not "part of" the E.U.. Let's try another way. We can guess that Europe is connected with the E.U. (in some way) and that the E.U. is connected to Italy. This is probably true in this way, reversed (Italy is connected to the E.U. and the E.U. is connected to Europe) or both.

In [9]:
# Apparently is not "part of" but something else. Count how many connections has Europe in general (considering that the thing connected has to also be connected with italy, in this case ?y are the possible candidate for the EU)
queryString = """
SELECT (count(?x) as ?n)
WHERE { 

wd:Q46 ?x ?y .
?y ?z wd:Q38 .
}  
"""

print("Results")
run_query(queryString)

Results
[('n', '1')]


1

In [6]:
# Show the possible properties
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 

wd:Q46 ?x ?y .
?y ?z wd:Q38 .
?x <http://schema.org/name> ?name .
}  
"""

print("Results")
run_query(queryString)

Results
[('x', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]


1

"has part" could be what we need to connect a country with the E.U.. Let's check.

In [7]:
# Now in the possible ?y we have to have the EU
queryString = """
SELECT ?y ?name
WHERE { 

?y wdt:P527 wd:Q38 .
?y <http://schema.org/name> ?name .
}  
"""

print("Results")
run_query(queryString)

Results
[('y', 'http://www.wikidata.org/entity/Q1072120'), ('name', 'Missile Technology Control Regime')]
[('y', 'http://www.wikidata.org/entity/Q1480793'), ('name', 'Nuclear Suppliers Group')]
[('y', 'http://www.wikidata.org/entity/Q1969730'), ('name', 'Schengen Area')]
[('y', 'http://www.wikidata.org/entity/Q19771'), ('name', 'G20')]
[('y', 'http://www.wikidata.org/entity/Q290712'), ('name', 'Statute of the Council of Europe')]
[('y', 'http://www.wikidata.org/entity/Q458'), ('name', 'European Union')]
[('y', 'http://www.wikidata.org/entity/Q782942'), ('name', 'Australia Group')]
[('y', 'http://www.wikidata.org/entity/Q8268'), ('name', 'Eurozone')]
[('y', 'http://www.wikidata.org/entity/Q27449'), ('name', 'Southern Europe')]
[('y', 'http://www.wikidata.org/entity/Q1769361'), ('name', 'list of countries with IKEA stores')]
[('y', 'http://www.wikidata.org/entity/Q30386'), ('name', 'Alpenländer')]
[('y', 'http://www.wikidata.org/entity/Q673158'), ('name', 'Wassenaar Arrangement')]
[('y',

18

Q458 represents the E.U.. Now let's check some connection to see if everything makes sense

In [12]:
# We found the EU, let's see if there are other connections with italy
queryString = """
SELECT ?y ?name
WHERE { 

wd:Q458 ?y wd:Q38 .
?y <http://schema.org/name> ?name .
}  
"""

print("Results")
run_query(queryString)

Results
[('y', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('y', 'http://www.wikidata.org/prop/direct/P150'), ('name', 'contains administrative territorial entity')]
[('y', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]


3

In [13]:
# "has part" is definitely what we need. EU has 27 state so check if it is correct
queryString = """
SELECT (count(?p) as ?x)
WHERE { 

wd:Q458 wdt:P527 ?p .

}  
"""

print("Results")
run_query(queryString)

Results
[('x', '27')]


1

We have everything that we need to specify the BGP to retrieve all the politicians in the E.U. countries.

In [17]:
# ANSWER to 1) Identify the BGP to retrieve E.U. countries and their politicians
queryString = """
SELECT ?politician ?country
WHERE { 
wd:Q458 wdt:P527 ?country .
?politician wdt:P106 wd:Q82955 .
?politician wdt:P27 ?country .
}limit 10
"""

print("Results")
run_query(queryString)

Results
[('politician', 'http://www.wikidata.org/entity/Q1930688'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q15727916'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q2718795'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q15845440'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q15845443'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q15845470'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q15850631'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q15891861'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q15925252'), ('country', 'http://www.wikid

10

***

# Task 2
*Identify the BGP for obtaining other occupations and properties of politicians*


We can also answer the second question about the BGP to obtain other occupations and properties of the politicians in the E.U..

In [23]:
# ANSWER to 2). Identify the BGP for obtaining other occupations and properties of politicians 
queryString = """
SELECT ?pol ?otherO ?otherP
WHERE { 
wd:Q458 wdt:P527 ?x .
?pol wdt:P106 wd:Q82955 .
?pol wdt:P27 ?x .
?pol wdt:P106 ?otherO .
?pol ?otherP ?a
filter (?otherO != wd:Q82955)
filter (?otherP != wdt:P106 )
}limit 10
"""
print("Results")
run_query(queryString)

Results
[('pol', 'http://www.wikidata.org/entity/Q1930688'), ('otherO', 'http://www.wikidata.org/entity/Q121594'), ('otherP', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type')]
[('pol', 'http://www.wikidata.org/entity/Q1930688'), ('otherO', 'http://www.wikidata.org/entity/Q188094'), ('otherP', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type')]
[('pol', 'http://www.wikidata.org/entity/Q1930688'), ('otherO', 'http://www.wikidata.org/entity/Q1930187'), ('otherP', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type')]
[('pol', 'http://www.wikidata.org/entity/Q1930688'), ('otherO', 'http://www.wikidata.org/entity/Q81096'), ('otherP', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type')]
[('pol', 'http://www.wikidata.org/entity/Q1930688'), ('otherO', 'http://www.wikidata.org/entity/Q121594'), ('otherP', 'http://www.wikidata.org/prop/direct-normalized/P1006')]
[('pol', 'http://www.wikidata.org/entity/Q1930688'), ('otherO', 'http://www.wikidata.org/entity/Q188094'), ('otherP', 'http://www.wikidat

10

***

# Task 3

*How many politicians are recorder for each E.U. country?*

Considering that we have the E.U. node and we know how is connected to the singles countries, we can also answer the third task.

In [8]:
# ANSWER to 3). How many politicians are recorder for each E.U. country?
queryString = """
SELECT (count(?politician) as ?npol) ?name
WHERE { 
wd:Q458 wdt:P527 ?country .
?politician wdt:P106 wd:Q82955 .
?politician wdt:P27 ?country .
?country <http://schema.org/name> ?name .
}group by (?name)
"""

print("Results")
run_query(queryString)

Results
[('npol', '1267'), ('name', 'Slovenia')]
[('npol', '5185'), ('name', 'Poland')]
[('npol', '1432'), ('name', 'Estonia')]
[('npol', '2237'), ('name', 'Czech Republic')]
[('npol', '791'), ('name', 'Netherlands')]
[('npol', '79874'), ('name', 'France')]
[('npol', '4802'), ('name', 'Greece')]
[('npol', '4111'), ('name', 'Hungary')]
[('npol', '30310'), ('name', 'Spain')]
[('npol', '7964'), ('name', 'Belgium')]
[('npol', '401'), ('name', 'Croatia')]
[('npol', '4170'), ('name', 'Finland')]
[('npol', '12028'), ('name', 'Italy')]
[('npol', '372'), ('name', 'Slovakia')]
[('npol', '1512'), ('name', 'Bulgaria')]
[('npol', '7912'), ('name', 'Sweden')]
[('npol', '2536'), ('name', 'Denmark')]
[('npol', '33613'), ('name', 'Germany')]
[('npol', '1092'), ('name', 'Latvia')]
[('npol', '222'), ('name', 'Malta')]
[('npol', '6997'), ('name', 'Austria')]
[('npol', '7'), ('name', 'Ireland')]
[('npol', '486'), ('name', 'Cyprus')]
[('npol', '1072'), ('name', 'Luxembourg')]
[('npol', '2201'), ('name', 'Po

27

# Random exploration

In the question is not specified, but maybe we have to consider only the alive ones, so let's try to find only them. We pick one at random and try to see if is dead or not (probably it will have a property about death).

In hindsight this wasn't a smart approach, it would have been smarter to check with a regex all the properties going out of politician.

In [3]:
# Let's try to find only the alive ones
queryString = """
SELECT ?politician ?country
WHERE { 
wd:Q458 wdt:P527 ?country .
?politician wdt:P106 wd:Q82955 .
?politician wdt:P27 ?country .
}limit 10
    """

print("Results")
run_query(queryString)

Results
[('politician', 'http://www.wikidata.org/entity/Q1930688'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q15727916'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q2718795'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q15845440'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q15845443'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q15845470'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q15850631'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q15891861'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q15925252'), ('country', 'http://www.wikid

10

In [5]:
# We are searching for something about death
queryString = """
SELECT distinct ?p ?name
WHERE { 
wd:Q1930688 ?p ?c .
?p <http://schema.org/name> ?name .

}
    """

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1045'), ('name', 'Sycomore ID')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1315'), ('name', 'NLA Trove ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by sour

77

There are few properties about death. Probably the better ones for our case (the one that probably will be present if someone is dead) are the date or the place. So we use that to refine the task 1 answer

In [2]:
# answer to 1 (with only the alive ones)
queryString = """
SELECT ?politician ?country
WHERE { 
wd:Q458 wdt:P527 ?country .
?politician wdt:P106 wd:Q82955 .
?politician wdt:P27 ?country .
filter not exists{?politician wdt:P570|wdt:P20 ?nnn}
}limit 10
"""

print("Results")
run_query(queryString)    

Results
[('politician', 'http://www.wikidata.org/entity/Q16007948'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q16011458'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q16011469'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q16025475'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q16026791'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q16026836'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q16026908'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q2898805'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('politician', 'http://www.wikidata.org/entity/Q528581'), ('country', 'http://www.wikida

10

***

# Task 4

*Are there politicians with double citizenship?*



In [12]:
# ANSWER to 4) Try to find politicians with more than one citizenship 
queryString = """
ask where {{
 select ?politician (count(?country) as ?c)
    WHERE { 
    wd:Q458 wdt:P527 ?country .
    ?politician wdt:P106 wd:Q82955 .
    ?politician wdt:P27 ?country .
    filter not exists{?politician wdt:P570 ?nnn} 
    } group by ?politician
    having (count(?country)>1)
  }  
}   
"""

print("Results")
run_ask_query(queryString) 

Results


{'head': {'link': []}, 'boolean': True}

***

# Task 5.1 and 5.2

*What are the top-3 occupations for a politician in Italy and France?*

*What if you consider only politicians for which we don't have a date of death?*

Let's start by finding the top occupations for a politician in Italy

In [13]:
# top occupations in italy
queryString = """
SELECT (count(?pol) as ?p) ?name
WHERE { 
?pol wdt:P106 wd:Q82955 .
?pol wdt:P27 wd:Q38 .
?pol wdt:P106 ?otherO .
filter (?otherO != wd:Q82955)
?otherO <http://schema.org/name> ?name .
}group by (?name)
order by desc (?p)
limit 100
"""

print("Results")
run_query(queryString) 

Results
[('p', '730'), ('name', 'journalist')]
[('p', '706'), ('name', 'lawyer')]
[('p', '337'), ('name', 'writer')]
[('p', '227'), ('name', 'entrepreneur')]
[('p', '226'), ('name', 'teacher')]
[('p', '220'), ('name', 'university teacher')]
[('p', '186'), ('name', 'trade unionist')]
[('p', '180'), ('name', 'economist')]
[('p', '173'), ('name', 'physician')]
[('p', '149'), ('name', 'engineer')]
[('p', '132'), ('name', 'jurist')]
[('p', '128'), ('name', 'historian')]
[('p', '107'), ('name', 'diplomat')]
[('p', '104'), ('name', 'magistrate')]
[('p', '80'), ('name', 'judge')]
[('p', '77'), ('name', 'businessperson')]
[('p', '69'), ('name', 'partisan')]
[('p', '66'), ('name', 'architect')]
[('p', '61'), ('name', 'philosopher')]
[('p', '58'), ('name', 'screenwriter')]
[('p', '55'), ('name', 'poet')]
[('p', '53'), ('name', 'civil servant')]
[('p', '45'), ('name', 'banker')]
[('p', '44'), ('name', 'military personnel')]
[('p', '42'), ('name', 'actor')]
[('p', '40'), ('name', 'television presen

100

We need the country of France obviously

In [29]:
# Find france
queryString = """
SELECT ?p ?name
WHERE { 

wd:Q458 wdt:P527 ?p .
?p <http://schema.org/name> ?name
}  
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('p', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('p', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia')]
[('p', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('p', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic')]
[('p', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('p', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia')]
[('p', 'http://www.wikidata.org/entity/Q218'), ('name', 'Romania')]
[('p', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria')]
[('p', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia')]
[('p', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('p', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('p', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary')]
[('p', 'http://www.wikidata.org/entity/Q29'), ('name', 'Spain')]
[('p', 'http://www.wikidata.org/entity/

27

Putting everything together for the answer of task 5. The occupation do not change even considering only the alive ones

In [18]:
# ANSWER to 5.1
queryString = """
SELECT distinct ?name ?p
WHERE{
    {
    SELECT (count(?pol) as ?p) ?name
    WHERE { 
        ?pol wdt:P106 wd:Q82955 .
        ?pol wdt:P27 wd:Q38 .
        ?pol wdt:P106 ?otherO .
        filter (?otherO != wd:Q82955)
        ?otherO <http://schema.org/name> ?name .
    }group by (?name)
    order by desc (?p)
    limit 3
    }
UNION
    {
    SELECT (count(?pol) as ?p) ?name
    WHERE { 
        ?pol wdt:P106 wd:Q82955 .
        ?pol wdt:P27 wd:Q142 .
        ?pol wdt:P106 ?otherO .
        filter (?otherO != wd:Q82955)
        ?otherO <http://schema.org/name> ?name .
    }group by (?name)
    order by desc (?p)
    limit 3
      }
}order by desc (?p)
"""

print("Results")
run_query(queryString)

Results
[('name', 'pensioner'), ('p', '14812')]
[('name', 'anciens cadres'), ('p', '5432')]
[('name', 'farm operator'), ('p', '4907')]
[('name', 'journalist'), ('p', '730')]
[('name', 'lawyer'), ('p', '706')]
[('name', 'writer'), ('p', '337')]


6

In [19]:
# ANSWER to 5.2
queryString = """
SELECT distinct ?name ?p
WHERE{
    {
    SELECT (count(?pol) as ?p) ?name
    WHERE { 
        ?pol wdt:P106 wd:Q82955 .
        ?pol wdt:P27 wd:Q38 .
        ?pol wdt:P106 ?otherO .
        filter not exists{?pol wdt:P570 ?nnn}
        filter (?otherO != wd:Q82955)
        ?otherO <http://schema.org/name> ?name .
    }group by (?name)
    order by desc (?p)
    limit 3
    }
UNION
    {
    SELECT (count(?pol) as ?p) ?name
    WHERE { 
        ?pol wdt:P106 wd:Q82955 .
        ?pol wdt:P27 wd:Q142 .
        ?pol wdt:P106 ?otherO .
        filter not exists{?pol wdt:P570 ?nnn}
        filter (?otherO != wd:Q82955)
        ?otherO <http://schema.org/name> ?name .
    }group by (?name)
    order by desc (?p)
    limit 3
    }
}order by desc (?p)
"""

print("Results")
run_query(queryString)

Results
[('name', 'pensioner'), ('p', '14701')]
[('name', 'anciens cadres'), ('p', '5432')]
[('name', 'farm operator'), ('p', '4893')]
[('name', 'journalist'), ('p', '330')]
[('name', 'lawyer'), ('p', '247')]
[('name', 'writer'), ('p', '131')]


6

***

# Task 5.3

*Which politicians had a spouse that was also a politician? How many in each country?*

First we need to find the property that connects the politician to his/her spouse

In [20]:
# Find the spouse property
queryString = """
SELECT distinct ?otherP ?name
WHERE { 
    wd:Q458 wdt:P527 ?country .
    ?politician wdt:P106 wd:Q82955 .
    ?politician wdt:P27 ?country .
    ?politician ?otherP ?a .
    ?otherP <http://schema.org/name> ?name
    filter (?otherP != wdt:P106 ) 
}limit 100
"""

print("Results")
run_query(queryString)

Results
[('otherP', 'http://www.wikidata.org/prop/direct/P4944'), ('name', 'Premiers préfets ID')]
[('otherP', 'http://www.wikidata.org/prop/direct/P6728'), ('name', 'Dictionnaire des Vendéens ID')]
[('otherP', 'http://www.wikidata.org/prop/direct/P4906'), ('name', 'Bargeton ID')]
[('otherP', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('otherP', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('otherP', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('otherP', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('otherP', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('otherP', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('otherP', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('otherP', 'http://www.wikidata.org/prop/direct/P1045'), ('n

100

In [39]:
# ANSWER to 5.3
queryString = """
SELECT (count(?politician) as ?p) ?name
WHERE { 
wd:Q458 wdt:P527 ?country .
?politician wdt:P106 wd:Q82955 .
?politician wdt:P27 ?country .
?politician wdt:P26 ?spouse . 
?spouse wdt:P106 wd:Q82955 .  
?country <http://schema.org/name> ?name .
}group by ?name
order by desc (?p)
"""

print("Results")
run_query(queryString)

Results
[('p', '294'), ('name', 'France')]
[('p', '221'), ('name', 'Germany')]
[('p', '163'), ('name', 'Spain')]
[('p', '95'), ('name', 'Sweden')]
[('p', '79'), ('name', 'Finland')]
[('p', '69'), ('name', 'Italy')]
[('p', '51'), ('name', 'Denmark')]
[('p', '44'), ('name', 'Poland')]
[('p', '43'), ('name', 'Hungary')]
[('p', '41'), ('name', 'Austria')]
[('p', '34'), ('name', 'Greece')]
[('p', '32'), ('name', 'Romania')]
[('p', '25'), ('name', 'Czech Republic')]
[('p', '22'), ('name', 'Estonia')]
[('p', '21'), ('name', 'Belgium')]
[('p', '18'), ('name', 'Portugal')]
[('p', '15'), ('name', 'Latvia')]
[('p', '14'), ('name', 'Slovenia')]
[('p', '10'), ('name', 'Cyprus')]
[('p', '10'), ('name', 'Lithuania')]
[('p', '9'), ('name', 'Luxembourg')]
[('p', '8'), ('name', 'Netherlands')]
[('p', '8'), ('name', 'Bulgaria')]
[('p', '6'), ('name', 'Croatia')]
[('p', '6'), ('name', 'Slovakia')]
[('p', '4'), ('name', 'Malta')]


26